In [ ]:
!pip install facenet_pytorch

In [ ]:
import cv2
import numpy as np
import torch
import argparse
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms
from facenet_pytorch.models.utils.detect_face import extract_face
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

In [ ]:
class Whitening(object):
    """
    Whitens the image.
    """
    def __call__(self, img):
        mean = img.mean()
        std = img.std()
        std_adj = std.clamp(min=1.0 / (float(img.numel()) ** 0.5))
        y = (img - mean) / std_adj
        return y

In [ ]:
class FaceFeaturesExtractor:
    def __init__(self):
        self.aligner = MTCNN(keep_all=True, thresholds=[0.6, 0.7, 0.9])
        self.facenet_preprocess = transforms.Compose([Whitening()])
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()

    def extract_features(self, img):
        bbs,_, landmarks = self.aligner.detect(img,landmarks=True)
        if bbs is None:
            # if no face is detected
            return (None, None, None)
        
        faces = torch.stack([extract_face(img, bb) for bb in bbs])
        embeddings = self.facenet(self.facenet_preprocess(faces)).detach().numpy()

        return (bbs, embeddings, landmarks)


In [ ]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    #img = load_img(file, target_size=(224,224))

    img = edge_detection(file)
    (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img = cv2.cvtColor(im_bw,cv2.COLOR_GRAY2RGB)
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)

    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

def crop_image(image):
    features_extractor = FaceFeaturesExtractor()
    bbs, embeddings, landmarks = features_extractor.extract_features(image)
    image_array = np.array(image, dtype=np.float32)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    bbs[:,1]-=5
    bbs[:,3]+=15
    crop_img = image_array[int(bbs[:,1][0]):int(bbs[:,3][0]), int(bbs[:,0][0]):int(bbs[:,2][0])]
    return crop_img

def edge_detection(image):
  scale = 1
  delta = 0
  ddepth = cv2.CV_16S
  src = cv2.imread(image, cv2.IMREAD_COLOR)
  src = crop_image(src)
  src = cv2.resize(src, (224,224), interpolation = cv2.INTER_AREA)
  src = cv2.GaussianBlur(src, (3, 3), 0)
  gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
  grad_x = cv2.Sobel(gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
  grad_y = cv2.Sobel(gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
    
    
  abs_grad_x = cv2.convertScaleAbs(grad_x)
  abs_grad_y = cv2.convertScaleAbs(grad_y)
    
    
  grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
  return grad


In [ ]:
test_image = 'Testing/test.png'
feat = extract_features(test_image,model)

In [14]:
import random
import os
import numpy as np

path = "/content/Final_Clusters/"
faces = {}

os.chdir(path)
with os.scandir(path) as files:
  for file in files:
    cluster_num = file.name
    subfolder_path = os.path.join(path, file)
    with os.scandir(subfolder_path) as sub_files:
      for sub_file in sub_files:
        if '.ipynb_checkpoints' not in sub_file.name:
          file_path = os.path.join(subfolder_path, sub_file)
          file_temp = np.load(file_path)
          faces[cluster_num] = file_temp

In [24]:
min_cluster = 0
min_dist = float("inf")

for face in faces:
  point1 = feat
  point2 = faces[face]
  dist = np.linalg.norm(point1 - point2)
  if dist<min_dist:
    min_cluster = face

In [ ]:
model_path = "/content/Final_Clusters/"+str(min_cluster)+"/best_model.pth"
config_path = "/content/Final_Clusters/"+str(min_cluster)+"/config.jscon"

In [ ]:
#@title
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!tts --list_models

In [ ]:
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer
use_cuda=False
config_path = None
speakers_file_path = None
language_ids_file_path = None
vocoder_path = None
vocoder_config_path = None
encoder_path = None
encoder_config_path = None
synthesizer = Synthesizer(
        model_path,
        config_path,
        speakers_file_path,
        language_ids_file_path,
        vocoder_path,
        vocoder_config_path,
        encoder_path,
        encoder_config_path,
        use_cuda,
    )
text = "I'm sorry, but I don't want to be an emperor."
wav = synthesizer.tts(
        text
    )
synthesizer.save_wav(wav, "/content/test.wav")